In [ ]:
import torch

import pybullet as p
import time
import numpy as np
import os
import sys
from collections import namedtuple
from net.DQN_Solver import DQNSolver
from net.env import daughter, get_state
#from net.fns import coord2angle

In [1]:
True or False

True

In [ ]:
def make_env(path2cfgs, angle_step=None, RTS = 1, centered = False):
    env = daughter(path2cfgs, angle_step=angle_step, RTS = RTS, centered=centered)
    env.init_enviroment()
    return env

def rad2deg(radians):
    return (radians/np.pi)* 180

def deg2rad(degrees):
    return (degrees/180) * np.pi

In [ ]:
# def robot(BATCH_SIZE, angle_step = None, path = './cfgs'):
# path = './cfgs'
# robot(500, 3)

path = './cfgs'
angle_step = 3
BATCH_SIZE = 50
CENTERED = False

env = make_env(path,angle_step=angle_step, centered = CENTERED)
body_index = 1; full = False

arms_and_legs = [29,35,37,42,48,50]

env.set_avalible_motors(arms_and_legs)
motors = env.mtrIdx if full else arms_and_legs

observation_space = get_state(body_index, motors).shape[0]
action_space      = len(motors)*2
dqn_solver        = DQNSolver(observation_space, 
                              action_space, 
                              BATCH_SIZE = BATCH_SIZE,
                              LEARNING_RATE = 0.01)

In [ ]:
env.reset()
p.setRealTimeSimulation(0)

In [ ]:
p.disconnect()
# p.setRealTimeSimulation(0)

In [ ]:
def det_single_motor_state(body_index, motor):
    array = []
    jointState = p.getJointState(body_index, motor)
    jointState = jointState[0],jointState[1],jointState[3]
    linkState  = p.getLinkState(body_index, motor)[:2]
    for source in [jointState, linkState]:
        for i in source:
            if not isinstance(i, (list, tuple)):
                array.append(i)
            else:
                array.extend(i)
    return array

def get_state(body_index, motor_list):
    array = []
    for motor in motor_list:
        state = det_single_motor_state(body_index, motor)
        array.extend(state)

    return torch.tensor(data = array, dtype = torch.float32).reshape(1,len(motor_list),-1)

In [ ]:
new_state = get_state(1, arms_and_legs)
new_state.shape

In [ ]:
bn = torch.nn.BatchNorm1d(6)
r = bn(new_state)
r

In [ ]:
r.shape

1. bodyUniqueId
2. linkIndex
3. computeLinkVelocity
4. computeForwardKinematics

link_id = 0
state = p.getLinkState(1,link_id,1,1)

1. linkWorldPosition (Cartesian position of center of mass)
2. linkWorldOrientation (Cartesian orientation of center of mass, 
   in quaternion [x,y,z,w])
3. localInertialFramePosition (local position offset of inertial frame 
   (center of mass) expressed in the URDF link frame)
4. localInertialFrameOrientation (local orientation (quaternion [x,y,z,w]) 
   offset of the inertial frame expressed in URDF link frame.)
5. worldLinkFramePosition (world position of the URDF link frame)
6. worldLinkFrameOrientation (world orientation of the URDF link frame)
7. worldLinkLinearVelocity (Cartesian world velocity. Only returned if 
   computeLinkVelocity non-zero.)
8. worldLinkAngularVelocity (Cartesian world velocity. Only returned if 
   computeLinkVelocity non-zero.)

In [ ]:
env.reset()
p.setRealTimeSimulation(0)

In [ ]:
env.reset()
p.setRealTimeSimulation(1)

In [ ]:
torso_angle = 15
legs_angle = 30
degrees = 0

pelvis, neck, foot = 0,8,[41,54] 

In [ ]:
pelvis_position = np.array(p.getLinkState(1, pelvis,1,1)[0])
neck_position   = np.array(p.getLinkState(1, neck,  1,1)[0])
foot_position   = landed_foot(foot)
coord2angle_N(pelvis_position, neck_position), coord2angle_N(foot_position, pelvis_position)

In [ ]:
degrees += 1
print(degrees)
force = 1000
maxVelocity = 360
p.setJointMotorControl2(
    1,
    3,
    p.POSITION_CONTROL,
    targetPosition = deg2rad(degrees),
    force          = force,
    maxVelocity    = deg2rad(maxVelocity)
)
# p.stepSimulation()

pelvis_position = np.array(p.getLinkState(1, pelvis,1,1)[0])
neck_position   = np.array(p.getLinkState(1, neck,  1,1)[0])
foot_position   = landed_foot(foot)
coord2angle_N(pelvis_position, neck_position), coord2angle_N(foot_position, pelvis_position)

In [ ]:
# def pelvis_foot_angle(pelvis = 0, foot_list = [41,54], annotated = False):
#     foot_position = landed_foot(foot_list)
#     pelvis_position = np.array(p.getLinkState(1, pelvis,1,1)[0])
#     if annotated:
#         return dict(pelvis_position=pelvis_position, foot_position=foot_position)
#     return foot_position, pelvis_position

# def pelvis_head_angle(pelvis = 0, neck = 8, annotated = False):
#     link_states = p.getLinkStates(1,[pelvis,neck],1,1)
#     positions = np.array([np.array(tuple_[0]) for tuple_ in link_states])
#     if annotated:
#         return dict(pelvis_position = positions[0], neck_position = positions[1])
#     return positions

# pf = pelvis_foot_angle(annotated = True)
# ph = pelvis_head_angle(annotated = True)
# pf, ph

In [ ]:
False*False

In [ ]:
minmax = [0, 90]
a = 910
if a>=minmax[0] and a<=minmax[1]:
    print(True)

In [ ]:
def midle_foot_coord(position):
    avg_pos = (position[0] + position[1])/2
    return avg_pos

def landed_foot(foot_idx_list = [41,54], threshold = 0.1):
    link_states = p.getLinkStates(1, foot_idx_list,1,1)
    position = np.array([np.array(tuple_[0]) for tuple_ in link_states])
    mask = [l[2]<threshold for l in position]
    position = position[mask]
    
    if position.shape[0]==2:
        position = midle_foot_coord(position)
    else:
        position = position[0]
    return position

import math
def rad2deg(radians: float) -> float:
    degree = radians*180/math.pi
    return degree

def deg2rad(degree: float) -> float:
    radians = degree*math.pi/180
    return radians

def coord2angle_N(point1: tuple, point2: tuple) -> float:
    angle = 0
    x = point2[0] - point1[0]
    y = point2[1] - point1[1]
    z = point2[2] - point1[2]
    
    xd = math.sqrt(math.pow(x,2)+math.pow(z,2))
    x_cos = z/xd
    x_arccos= math.acos(x_cos)
    x_angle = rad2deg(x_arccos)
    
    yd = math.sqrt(math.pow(y,2)+math.pow(z,2))
    y_cos = z/yd
    y_arccos= math.acos(y_cos)
    y_angle = rad2deg(y_arccos)
    
    xyd  = math.sqrt(math.pow(x,2)+math.pow(y,2))
    xyzd = math.sqrt(math.pow(z,2)+math.pow(xyd,2))
    xyz_cos   = z/xyzd
    xyz_arccos= math.acos(xyz_cos)
    xyz_angle = rad2deg(xyz_arccos)
    
    return round(x_angle,1),round(y_angle,1),round(xyz_angle,1)

In [ ]:
pelvis, neck, foot = 0,8,[41,54] 
pelvis_position = np.array(p.getLinkState(1, pelvis,1,1)[0])
neck_position   = np.array(p.getLinkState(1, neck,  1,1)[0])
foot_position   = landed_foot(foot)
pelvis_position, neck_position, foot_position

In [ ]:
neck_position - pelvis_position

In [ ]:
pelvis_position - foot_position

In [ ]:
base_angle = coord2angle_N(pelvis_position, neck_position), coord2angle_N(foot_position, pelvis_position)

In [ ]:
p.getBasePositionAndOrientation(1)

In [ ]:
p.getLinkState(1,0,1,1)[0]

In [ ]:
a = 3 
b = 4

In [ ]:
a==4 or b ==4